In [ ]:
# main.py


from simulation import build_hamiltonian, cut_circuit, run_exact_sampler, reconstruct_expectation, trotterization_circuit
from utils import circuit_data_dict, save_to_json

from qiskit.transpiler import CouplingMap
from qiskit.quantum_info import SparsePauliOp
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def main(config):

    # Define Copling map for qubits and total number of qubits
    num_qubits = config.NUM_RINGS * config.NUM_SPINS
    edges = config.EDGES
    coupling_map = CouplingMap(edges)

    # Build Hamiltonian
    H = build_hamiltonian(
        num_qubits=num_qubits,
        coupling_map=coupling_map,
        single_gates=config.SINGLE_GATES_LIST,
        two_gates=config.TWO_GATES_LIST,
        anisotropy=config.ANISOTROPY,
        h=config.H_FIELD
    )

    #Obtain evolution circuit under TrotterLie approximation
    evolutionQC = trotterization_circuit(hamiltonian = H, trotter_reps = config.TROTTER_REPS, dt = config.DT)

    # Define Observables (for now Z on each site, ZII + IZI + IIZ)
    z_list = [('Z', [i], 1.0) for i in range(num_qubits)]
    z_observables = SparsePauliOp.from_sparse_list(z_list, num_qubits=num_qubits)

    # Cut circuit and generate subexperiments
    cutting_labels = config.NUM_SPINS * 'A' + config.NUM_SPINS * 'B' #Labels that mark the cutting point. 
    subexperiments, coefficients, subobservables = cut_circuit(
        circuit=evolutionQC,
        partition_labels=cutting_labels,
        observables=z_observables,
        num_samples=config.N_SAMPLES
    )

    # Run Simulations
    results = run_exact_sampler(subexperiments)

    # Reconstruct Expectation Values
    reconstructed_expval = reconstruct_expectation(
        results, coefficients, subobservables, z_observables
    )

    # Generate experiment and subcircuit data dictionary, and save to JSON
    basis_gates = ["h", "rx", "ry", "rz", "rxx", "rzz", "ryy", "cx"]
    data_dict = circuit_data_dict(subexperiments, coefficients, reconstructed_expval, basis_gates)
    save_to_json(data_dict)

class Config:
    #Class containing the configuration parameters
    def __init__(self):
        # Hamiltonian parameters
        self.NUM_SPINS = 12  # Number of qubits per ring
        self.NUM_RINGS = 2  # Total number of rings 
        self.ANISOTROPY = 1.0
        self.H_FIELD = 1.0
       
        # Gates in the Hamiltonian
        self.TWO_GATES_LIST = ['XX']
        self.SINGLE_GATES_LIST = ['Z']
        
        # Hamiltonian coupling map edges
        self.EDGES = [
            (ring * self.NUM_SPINS + i, ring * self.NUM_SPINS + (i + 1) % self.NUM_SPINS)
            for ring in range(self.NUM_RINGS)
            for i in range(self.NUM_SPINS)
        ] + [(0, 2 * self.NUM_SPINS - 1)]  # The coupling map is unidirectional. Does this affect dynamics?

        #Evolution parameters
        self.TROTTER_REPS = 2
        self.DT = 0.1  # Time parameter


        # Sampling
        self.N_SAMPLES = np.inf
        # self.N_SAMPLES = 10**6



if __name__ == "__main__":
    
    config = Config()

    main(config)


Subcircuits data has been successfully saved to 'subcircuits.json'.
